In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [2]:
import pandas as pd
import numpy as np
import nltk
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
data_path = "/home/hongphuc95/notebookteam/dataset/"

In [4]:
api_path = "/home/hongphuc95/notebookteam/api/"

In [5]:
# Load the movielens-100k dataset (download it if needed).
#data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
#algo = SVD()


In [6]:
business_df = pd.read_json(data_path + "business.json", lines=True)

In [7]:
review_df = pd.read_csv("/home/hongphuc95/notebookteam/fu/review.csv")

In [8]:
review_df.shape

(3494121, 9)

In [9]:
n_active = 40
user_review_df_count = review_df.groupby("user_id").size()
user_review_active_values = user_review_df_count[user_review_df_count >= n_active].reset_index()[
    "user_id"].values
user_active_df = review_df[review_df["user_id"].isin(user_review_active_values)]
user_active_df = user_active_df.groupby(["user_id", "business_id"], as_index=False).mean()

In [10]:
user_active_df.shape

(519838, 6)

In [11]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(review_df[['user_id', 'business_id', 'stars']], reader)

In [12]:
from surprise.model_selection import GridSearchCV
from surprise import NormalPredictor

In [13]:
param_grid = {'n_factors':[20,30,50,100]}

In [14]:
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5, n_jobs=-1)
gs.fit(data)
params = gs.best_params['rmse']

/home/hongphuc95/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [15]:
print("Best number of factors: %d" % (params['n_factors']))

Best number of factors: 20


In [18]:
svdtuned = SVD(n_factors=params['n_factors'])

In [19]:
# Run 5-fold cross-validation and print results.
cross_validate(svdtuned, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3276  1.3269  1.3283  1.3280  1.3254  1.3273  0.0010  
MAE (testset)     1.0689  1.0678  1.0689  1.0685  1.0667  1.0682  0.0008  
Fit time          70.20   72.28   70.61   71.76   70.22   71.02   0.85    
Test time         11.69   8.21    8.37    8.51    8.08    8.97    1.37    


{'test_rmse': array([1.32763919, 1.32694807, 1.32827826, 1.32802855, 1.3254026 ]),
 'test_mae': array([1.06891823, 1.06779658, 1.06889518, 1.06854169, 1.06672802]),
 'fit_time': (70.20356464385986,
  72.27842116355896,
  70.61429762840271,
  71.76304912567139,
  70.21897721290588),
 'test_time': (11.692693948745728,
  8.206737279891968,
  8.372921705245972,
  8.507348775863647,
  8.079296350479126)}

In [1]:
from geopy.exc import GeocoderTimedOut, GeocoderServiceError, GeocoderUnavailable
from geopy.geocoders import Nominatim

In [4]:
geolocator = Nominatim(user_agent="Test_Agent")
location = geolocator.reverse("52.509669, 13.376294")

In [17]:
location.raw["address"]["city"]

'Berlin'